In [ ]:
!pip install -q transformers datasets scikit-learn accelerate

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tweets-test_GT.csv to tweets-test_GT.csv
Saving tweets-test_PT.csv to tweets-test_PT.csv
Saving tweets-train_GT.csv to tweets-train_GT.csv
Saving tweets-train_PT.csv to tweets-train_PT.csv
Saving tweets-valid_GT.csv to tweets-valid_GT.csv
Saving tweets-valid_PT.csv to tweets-valid_PT.csv


In [ ]:
pt_train_df = pd.read_csv("tweets-train_PT.csv")
pt_val_df = pd.read_csv("tweets-valid_PT.csv")
pt_test_df = pd.read_csv('tweets-test_PT.csv')
gt_train_df = pd.read_csv("tweets-train_GT.csv")
gt_val_df = pd.read_csv("tweets-valid_GT.csv")
gt_test_df = pd.read_csv('tweets-test_GT.csv')

In [ ]:
print(pt_train_df.head())
print("\n")
print(gt_train_df.head())

                                               tweet  label
0  ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...     -1
1  सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...     -1
2  उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...     -1
3  आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...      1
4  बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...     -1


                                               tweet  label  political
0          होता होता राहीलेला  निवडणूक मारो मर्ज़ीभई     -1          0
1                         खरा लखोबा तर हा बोबडाच आहे     -1         -1
2  मुंबईतील घाटकोपरमध्ये धुळवड खेळून घरी परतलेलं ...     -1          0
3      वेडाबाई भूतकाळ बघ लोक शेन घालतात आणी दांडा ही     -1         -1
4  कुर्ला वाहतुक विभागाला फक्त हे पाठवले जाते पण ...     -1         -1


In [ ]:
# Standardize column names by renaming text fields to 'text'

#Training
pt_train_df = pt_train_df.rename(columns={"tweet": "text"})
gt_train_df = gt_train_df.rename(columns={"tweet": "text"})
pt_train_df = pt_train_df[["text", "label"]]
gt_train_df = gt_train_df[["text", "label"]]

# Combine MR and PT datasets into a single train dataframe
train_df = pd.concat([pt_train_df, gt_train_df], ignore_index=True)

# Shuffle the datasets to avoid ordering bias during training
train_df = train_df.sample(frac=1, random_state=42)

In [ ]:
#similarly for validation and test
# Validation
pt_val_df = pt_val_df.rename(columns={"tweet": "text"})
gt_val_df = gt_val_df.rename(columns={"tweet": "text"})
val_df = pd.concat([pt_val_df[["text", "label"]], gt_val_df[["text", "label"]]], ignore_index=True)
val_df = val_df.sample(frac=1, random_state=42)
# Test
pt_test_df = pt_test_df.rename(columns={"tweet": "text"})
gt_test_df = gt_test_df.rename(columns={"tweet": "text"})
test_df = pd.concat([pt_test_df[["text", "label"]], gt_test_df[["text", "label"]]], ignore_index=True)
test_df = test_df.sample(frac=1, random_state=42)

In [ ]:
# Convert original sentiment labels to numeric IDs for model training
# Also ensure 'text' column contains only strings and handle NaN values

for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].astype(int)
    df["label"] = df["label"] + 1  # -1->0, 0->1, 1->2
    df["text"] = df["text"].astype(str).fillna("") # Ensure text is string and handle NaNs

In [ ]:
# Convert pandas dataframes into HuggingFace Dataset objects

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

In [ ]:
# Tokenize text: pad, truncate, and encode inputs for BERT

def tokenize_fn(batch):
    # Ensure each text item is explicitly a string within a list for tokenizer compatibility
    texts = [str(x) for x in batch["text"]]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

In [ ]:
# Load Marathi-BERT-v2 tokenizer and model for sequence classification

model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: pos/neg/neutral

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encoded = train_dataset.map(tokenize_fn, batched=True)
val_encoded   = val_dataset.map(tokenize_fn, batched=True)
test_encoded  = test_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/24114 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3750 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define accuracy, precision, recall, and F1 score metrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

In [ ]:
# Configure training parameters such as epochs, batch size, and learning rate

training_args = TrainingArguments(
    output_dir="./mr_gt_marathi_bert_v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    fp16=True,             # if your GPU supports it
    report_to="none"
)


In [ ]:
# Initialize Trainer and begin fine-tuning Marathi-BERT-v2

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-3285206523.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.484300,0.444877,0.828667,0.834416,0.828667,0.830220
2,0.403500,0.431123,0.834000,0.836331,0.834000,0.834784
3,0.276500,0.465714,0.834333,0.834339,0.834333,0.834323


TrainOutput(global_step=4524, training_loss=0.45187400738609035, metrics={'train_runtime': 847.0799, 'train_samples_per_second': 85.402, 'train_steps_per_second': 5.341, 'total_flos': 4758537716319744.0, 'train_loss': 0.45187400738609035, 'epoch': 3.0})

In [ ]:
# Evaluate the fine-tuned model on the combined test set

test_metrics = trainer.evaluate(test_encoded)
print("Test metrics:", test_metrics)


Test metrics: {'eval_loss': 0.47434109449386597, 'eval_accuracy': 0.8264, 'eval_precision_macro': 0.8259828328502046, 'eval_recall_macro': 0.8264, 'eval_f1_macro': 0.8260254907113266, 'eval_runtime': 7.5095, 'eval_samples_per_second': 499.367, 'eval_steps_per_second': 31.294, 'epoch': 3.0}


In [ ]:
pt_test_df_processed = pt_test_df.copy()
pt_test_df_processed['label'] = pt_test_df_processed['label'].astype(int)
pt_test_df_processed['label'] = pt_test_df_processed['label'] + 1 # -1->0, 0->1, 1->2
pt_test_df_processed['text'] = pt_test_df_processed['text'].astype(str).fillna('')
pt_test_dataset = Dataset.from_pandas(pt_test_df_processed)
pt_test_encoded = pt_test_dataset.map(tokenize_fn, batched=True)

pt_test_metrics = trainer.evaluate(pt_test_encoded)
print("MahaSent_MR Test metrics:", pt_test_metrics)

Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

MahaSent_MR Test metrics: {'eval_loss': 0.4058034420013428, 'eval_accuracy': 0.8506666666666667, 'eval_precision_macro': 0.8510521378389707, 'eval_recall_macro': 0.8506666666666666, 'eval_f1_macro': 0.8504335150754297, 'eval_runtime': 4.6694, 'eval_samples_per_second': 481.861, 'eval_steps_per_second': 30.197, 'epoch': 3.0}


In [ ]:
gt_test_df_processed = gt_test_df.copy()
gt_test_df_processed['label'] = gt_test_df_processed['label'].astype(int)
gt_test_df_processed['label'] = gt_test_df_processed['label'] + 1 # -1->0, 0->1, 1->2
gt_test_df_processed['text'] = gt_test_df_processed['text'].astype(str).fillna('')
gt_test_dataset = Dataset.from_pandas(gt_test_df_processed)
gt_test_encoded = gt_test_dataset.map(tokenize_fn, batched=True)

gt_test_metrics = trainer.evaluate(gt_test_encoded)
print("MahaSent_GT Test metrics:", gt_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_GT Test metrics: {'eval_loss': 0.5771473050117493, 'eval_accuracy': 0.79, 'eval_precision_macro': 0.7900277856776189, 'eval_recall_macro': 0.79, 'eval_f1_macro': 0.7899945454910583, 'eval_runtime': 3.1352, 'eval_samples_per_second': 478.435, 'eval_steps_per_second': 29.982, 'epoch': 3.0}


In [ ]:
uploaded = files.upload()

Saving MahaSent_MR_Test.csv to MahaSent_MR_Test.csv
Saving MahaSent_ST_Test.csv to MahaSent_ST_Test.csv


In [ ]:
st_test_df = pd.read_csv('MahaSent_ST_Test.csv')
print("MahaSent_ST Test DataFrame loaded successfully.")
print(st_test_df.head())

MahaSent_ST Test DataFrame loaded successfully.
   Unnamed: 0                                       marathi_text  label
0           0                                       मोनिका नाही!     -1
1           1  मला भीती वाटते की परिस्थिती आमच्या अपेक्षेपेक्...     -1
2           2            अरे, तुमच्याकडे आधीपासून सर्वकाही नाही.     -1
3           3                  जॉय, तिच्यावर मारहाण करणे थांबवा.     -1
4           4                                 अरे देवा, हे बघ...     -1


In [ ]:
st_test_df_processed = st_test_df.copy()
st_test_df_processed = st_test_df_processed.rename(columns={"marathi_text": "text"})
st_test_df_processed['label'] = st_test_df_processed['label'].astype(int)
st_test_df_processed['label'] = st_test_df_processed['label'] + 1 # -1->0, 0->1, 1->2
st_test_df_processed['text'] = st_test_df_processed['text'].astype(str).fillna('')
st_test_dataset = Dataset.from_pandas(st_test_df_processed)
st_test_encoded = st_test_dataset.map(tokenize_fn, batched=True)

print("MahaSent_ST Test DataFrame processed and encoded successfully.")

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_ST Test DataFrame processed and encoded successfully.


In [ ]:
st_test_metrics = trainer.evaluate(st_test_encoded)
print("MahaSent_ST Test metrics:", st_test_metrics)

MahaSent_ST Test metrics: {'eval_loss': 0.8449007272720337, 'eval_accuracy': 0.7073333333333334, 'eval_precision_macro': 0.7681717559554725, 'eval_recall_macro': 0.7073333333333333, 'eval_f1_macro': 0.7075114617678314, 'eval_runtime': 3.2133, 'eval_samples_per_second': 466.805, 'eval_steps_per_second': 29.253, 'epoch': 3.0}


In [ ]:
mr_test_df = pd.read_csv('MahaSent_MR_Test.csv')
print("MahaSent_PT Test DataFrame loaded successfully.")
print(mr_test_df.head())

MahaSent_PT Test DataFrame loaded successfully.
   Unnamed: 0                                   marathi_sentence  label
0           0  जयदीप त्याच्या आई संहिता हिच्या घटस्फोटाचा भार...     -1
1           1                  मात्र तो त्यात यशस्वी झालेला नाही     -1
2           2  जेव्हा लोकांचा समूह त्यांच्या ड्रॉईंग रूमच्या ...     -1
3           3  दिग्दर्शकाची चित्रपटावरची पकडच पक्की नसल्यानं ...     -1
4           4  तो झिपऱ्याचे जग उत्तम प्रकारे निर्माण करत असता...     -1


In [ ]:
mr_test_df_processed = mr_test_df.copy()
mr_test_df_processed = mr_test_df_processed.rename(columns={"marathi_sentence": "text"})
mr_test_df_processed['label'] = mr_test_df_processed['label'].astype(int)
mr_test_df_processed['label'] = mr_test_df_processed['label'] + 1
mr_test_df_processed['text'] = mr_test_df_processed['text'].astype(str).fillna('')
mr_test_dataset = Dataset.from_pandas(mr_test_df_processed)
mr_test_encoded = mr_test_dataset.map(tokenize_fn, batched=True)
print("MahaSent_MR Test DataFrame processed and encoded successfully.")

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_MR Test DataFrame processed and encoded successfully.


In [ ]:
mr_test_metrics = trainer.evaluate(mr_test_encoded)
print("MahaSent_MR Test metrics:", mr_test_metrics)

MahaSent_MR Test metrics: {'eval_loss': 0.8532402515411377, 'eval_accuracy': 0.7146666666666667, 'eval_precision_macro': 0.7186582656197406, 'eval_recall_macro': 0.7146666666666667, 'eval_f1_macro': 0.7129762079262894, 'eval_runtime': 8.2663, 'eval_samples_per_second': 181.459, 'eval_steps_per_second': 11.371, 'epoch': 3.0}
